In [ ]:
# Import necessary libraries
# Import the required libraries
import boto3
from botocore.exceptions import ClientError

import json
import os
import io
from PIL import Image
import sagemaker
import pandas as pd
import numpy as np
import re

from keras import utils as kutils

In [ ]:
import boto3
sts = boto3.client('sts')
print(sts.get_caller_identity())

In [ ]:
def _get_default_session():
    if DEFAULT_SESSION is None:
        setup_default_session()
    return DEFAULT_SESSION

def setup_default_session(**kwargs):
    DEFAULT_SESSION = Session(**kwargs)

In [ ]:
s3_bucket   = 's3-avalanche-guard'
s3_key      = 'data/cv/uibk/images/none/2015-01-01 hafelekar (3).jpg' # Can be a nested key/file.
aws_profile = ''
aws_region  = 'us-west-1' # N California

aws_access_key_id = ''
aws_secret_access_key = ''

In [ ]:
import base64

def read_image_from_s3(bucket, key, region_name):
    """Load image file from s3.

    Parameters
    ----------
    bucket: string
        Bucket name
    key : string
        Path in s3

    Returns
    -------
    np array
        Image array
    """
    
    
    s3 = boto3.resource('s3', region_name)
    bucket = s3.Bucket(bucket)
    object = bucket.Object(key)
    response = object.get()
    file_stream = response['Body']
    
   
    im = Image.open(file_stream)
   
    
    '''
    # Encode the image as a base64-encoded string
    img_b64 = base64.b64encode(im).decode('utf-8')
    # Convert the base64-encoded string back to bytes
    img_bytes = bytes(img_b64, 'utf-8')
    '''

    with io.BytesIO() as buf:
        im.save(buf, 'jpeg')
        img_bytes = buf.getvalue()
    
    return np.array(im),  im, im, img_bytes

In [8]:
image_arry, image, im_b64, im_bytes = read_image_from_s3(s3_bucket, s3_key, region_name = aws_region) 





In [ ]:
# The name of the SageMaker endpoint running in our account.  
# Note this needs to be in the same region as this running code. 
client = boto3.client('sagemaker-runtime')
endpointName = 'jumpstart-dft-resnet-v2-152-classification-4'
endpointName = 'tensorflow-inference-2024-07-07-23-54-28-904'
print(client)

In [ ]:
import os
cwd = os.getcwd()
print(cwd)

In [ ]:
class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)



In [ ]:
# Load the binary data of each image into a Python variable.
f = open('./AR/images/local_image_directory/test/positive/2012-05-02 bremer huette (1).jpg', 'rb') # opening a binary file
#data = im_bytes ##f.read()
data = f.read()

image_size = (224,224)
img_path = './AR/images/local_image_directory/test/positive/2012-05-02 bremer huette (1).jpg'

#fig.tight_layout()
img = kutils.load_img(img_path, target_size=image_size)
#plt.imshow(img)

img_array = kutils.img_to_array(img)
img_array = np.expand_dims(img_array, 0)  # Create batch axis


##data= img_array.to_json()
data_json = 
data = {"instances": [data_json]} 

body_content = base64.b64decode(data)
# Call the endpoint with the image in memory. The endpoint is invoked and a result is recorded.
response = client.invoke_endpoint(
    EndpointName=endpointName, #ContentType="application/x-image",
    ContentType=  'application/json',  #'text/csv',
    Body=
)

#print(response)

result = json.loads(response['Body'].read().decode())

print(result) 

In [ ]:
from PIL import Image
import numpy as np
import json
import boto3
from keras import backend as K

client = boto3.client('sagemaker-runtime') 
img_path = './AR/images/local_image_directory/test/positive/2012-05-02 bremer huette (1).jpg'

body = json.dumps({"instances": image.tolist()})
ioc_predictor_endpoint_name  = 'tensorflow-inference-2024-07-07-23-54-28-904'
content_type = 'application/json'   
#sess=K.get_session()


endpoint_name = ioc_predictor_endpoint_name                                      # Your endpoint name.
# The MIME type of the input data in the request body.
accept = content_type                                              # The desired MIME type of the inference in the response.
payload = body                                           # Payload for inference.
response = client.invoke_endpoint(
    EndpointName=endpoint_name, 
    ContentType=content_type,
    Accept=accept,
    Body=payload
    )

'''
ioc_response = client.invoke_endpoint(
    
    EndpointName=ioc_predictor_endpoint_name,
    Body=body,
    ContentType=content_type
 )
'''